## Library Imports

In [16]:
from scipy.signal import lfilter
from scipy.signal import butter
import IPython.display as ipd
import librosa.display
import pandas as pd
import numpy as np
import speechpy
import tsfel
import os

## Utility Functions

### Function for Bandpass Filter

In [17]:
# Creating Butterworth Bandpass Filter
def butter_bandpass(lowcut : int, highcut : int, fs : int, order=5):
    """ Create a Butterworth Bandpass Filter

        \tlowcut: low cut frequency (in Hz)
        \thighcut: high cut frequency (in Hz)
        \tfs: sampling frequency (in Hz)
        \torder: order of the filter (default = 5)

        return: b, a: numerator (b) and denominator (a) polynomials of the IIR filter   """

    nyq = 0.5 * fs                                      # Nyquist frequency (half of the sampling frequency)
    low = lowcut / nyq                                  # Normalized low cut frequency
    high = highcut / nyq                                # Normalized high cut frequency
    b, a = butter(order, [low, high], btype='band')     # Create Butterworth bandpass filter
    return b, a                                         # Return numerator (b) and denominator (a) polynomials of the IIR filter


def butter_bandpass_filter(data, lowcut : int, highcut : int, fs : int, order=5):
    """     Apply a Butterworth Bandpass Filter   
            
            \tdata: signal to be filtered
            \tlowcut: low cut frequency (in Hz)
            \thighcut: high cut frequency (in Hz)
            \tfs: sampling frequency (in Hz)
            \torder: order of the filter (default = 5)

            return: y: filtered signal   """


    b, a = butter_bandpass(lowcut, highcut, fs, order=order)    # Create Butterworth bandpass filter
    y = lfilter(b, a, data)                                     # Apply filter
    return y                                                    # Return filtered signal

### Function for Finding Peak Landmarks (Start, End, and Peak)

In [18]:
def find_start_peak_end(x_filt,windowsize=100):
    """     Find the start, peak and end of the signal
            \tx_filt: filtered signal
            \twindowsize: size of the window to calculate the standard deviation (default = 100)
            return: start, peak, end: start, peak and end of the signal   """

    peak = np.argmax(np.abs(x_filt))    # Find the peak of the signal
    start=0                             # Initialize start 
    end = len(x_filt)                   # Initialize end

    std_cutoff = np.std(x_filt)/10      # Set the standard deviation cutoff

    # Find the start and end of the signal
    for i in range(peak,0,-windowsize):  

        # Calculate the standard deviation of the signal in the window
        std = np.std(x_filt[i:i+windowsize])

        # If the standard deviation is less than the cutoff, set the start of the signal
        if std <= std_cutoff:
            start = i
            break

    # Find the end of the signal
    for j in range(peak,len(x_filt),windowsize):

        # Calculate the standard deviation of the signal in the window
        std = np.std(x_filt[j:j+windowsize])

        # If the standard deviation is less than the cutoff, set the end of the signal
        if std <= std_cutoff:
            end = j
            break
    
    # Sanity check so that end doesn't go out of bounds
    if end==len(x_filt):
        end = len(x_filt)-3201
    
    # Adding 0.1s of margin to end 
    return start,peak,end+3200

### Function to Extract Features

In [19]:

def Audio_Feature_Extractor(directory='./N95/',build_header=True,build_spectrogram=True,build_mfc=True,build_melspectogram=True,build_mfe=True,build_mfcc=True,build_spectral_features=True,build_temporal_features=True,Apply_Bandpass_Filter=False,cutoff_low=1250,cutoff_high=2500,order=5,Apply_clipping=False,Auto_Clipping=False,clip_start=12800,clip_end=48000, Normalize=False):

    """ 
    This function extracts the features from the audio files in the given directory.\n
    The features extracted are:\n
        1. Spectrogram (STFT) (uses librosa library for extraction)
        2. MFC (uses librosa library for extraction)
        3. Mel Spectrogram  (uses librosa library for extraction)
        4. MFE  (uses speechpy library for extraction)
        5. MFCC  (uses speechpy library for extraction)
        6. MFCC CMVN  (uses speechpy library for extraction)
        7. Spectral Features  (uses tsfel library for extraction)
        8. Temporal Features  (uses tsfel library for extraction)

    Parameters:
        directory (str): Path to the directory containing the audio files.
        build_header (bool): If True, the header is built.
        build_spectrogram (bool): If True, the spectrogram is built.
        build_mfc (bool): If True, the MFC is built.
        build_melspectogram (bool): If True, the mel spectrogram is built.
        build_mfe (bool): If True, the MFE is built.
        build_mfcc (bool): If True, the MFCC is built.
        build_spectral_features (bool): If True, the spectral features are built.
        build_temporal_features (bool): If True, the temporal features are built.
        Apply_Bandpass_Filter (bool): If True, the bandpass filter is applied.
        cutoff_low (int): Lower cutoff frequency for the bandpass filter.
        cutoff_high (int): Higher cutoff frequency for the bandpass filter.
        order (int): Order of the bandpass filter.
        Apply_clipping (bool): If True, the audio is clipped.
        clip_start (int): Starting point of the audio clip.
        clip_end (int): Ending point of the audio clip.
        Normalize (bool): If True, the wave is normalised before extracting features.
        
    """

    all_features = []
    header = ['pid']
    i=0
    
    # Toggle through all the files in the directory.
    for filename in os.listdir(directory):

        # Path to the file.
        f = os.path.join(directory, filename)

        # checking if it is a file
        if os.path.isfile(f):
            
            i=i+1
            print(i,'/',len(os.listdir(directory)))

            curr_feature = np.array([])

            # Loading the audio file
            x , sr = librosa.load(f,sr=16000)


            if Normalize:
                x = x/np.max(np.abs(x))

            # Applying bandpass filter
            if Apply_Bandpass_Filter:
                x = butter_bandpass_filter(x,cutoff_low,cutoff_high,sr,order)

            # Normalizing the wave
            if Normalize:
                x = x/np.max(np.abs(x))

            # Applying clipping
            if Apply_clipping:
                if Auto_Clipping:
                    start,peak,end = find_start_peak_end(x)
                    x = x[start:end]
                else:
                    x = x[clip_start:clip_end]

            # Retrieving the patient ID from the filename.
            pid = np.array([filename.split('.')[0]])
            curr_feature = np.concatenate((curr_feature,pid))

            # Extracting the features.
            num_filters     =   40
            num_cepstral    =   5 
            fft_length      =   512

            # Spectrogram of the audio file.
            if build_spectrogram:
                stft_spectrogram=np.mean(np.abs(librosa.stft(x, n_fft=480, hop_length=240)),axis=1)
                curr_feature = np.concatenate((curr_feature,stft_spectrogram))
                if build_header:
                    header.extend(['stft_spectrogram_'+str(i) for i in range(stft_spectrogram.shape[0])])

            # MFC coeff of the audio file.
            if build_mfc:
                mfc_coefficients = np.mean(librosa.feature.mfcc(y=x, n_mfcc=50, sr=16000),axis=1)
                curr_feature = np.concatenate((curr_feature,mfc_coefficients))
                if build_header:
                    header.extend(['mfc_coefficients_'+str(i) for i in range(mfc_coefficients.shape[0])])

            # Mel Spectrogram of the audio file.
            if build_melspectogram:
                melspectogram = np.mean(librosa.feature.melspectrogram(y=x, sr=16000, n_mels=16, n_fft=480, hop_length=240),axis=1)
                curr_feature = np.concatenate((curr_feature,melspectogram))
                if build_header:
                    header.extend(['melspectogram_'+str(i) for i in range(melspectogram.shape[0])])

            # MFE of the audio file.
            if build_mfe:
                mfe = np.mean(speechpy.feature.mfe(x, sampling_frequency=16000, frame_length=0.030, frame_stride=0.015,num_filters=num_filters,fft_length=fft_length,low_frequency=0)[0],axis=0)
                curr_feature = np.concatenate((curr_feature,mfe))
                if build_header:
                    header.extend(['mfe_'+str(i) for i in range(mfe.shape[0])])

            # MFCC and MFCC CMVN of the audio file.
            if build_mfcc:
                mfcc = speechpy.feature.mfcc(x, sampling_frequency=16000, frame_length=0.030, frame_stride=0.015,num_filters=num_filters, fft_length=fft_length, low_frequency=0, num_cepstral=num_cepstral)
        
                mfcc_cmvn = np.mean(speechpy.processing.cmvnw(mfcc,win_size=301,variance_normalization=True), axis=0)
                mfcc = np.mean(mfcc,axis=0)
                curr_feature = np.concatenate((curr_feature,mfcc))
                curr_feature = np.concatenate((curr_feature,mfcc_cmvn))
                if build_header:
                    header.extend(['mfcc_'+str(i) for i in range(mfcc.shape[0])])
                    header.extend(['mfcc_cmvn_'+str(i) for i in range(mfcc_cmvn.shape[0])])

            # Spectral features of the audio file.
            if build_spectral_features:
                cfg = tsfel.get_features_by_domain("spectral")
                spectral_features = np.array(tsfel.time_series_features_extractor(cfg, x, fs=16000, window=320,verbose=0))[0]
                curr_feature = np.concatenate((curr_feature,spectral_features))
                if build_header:
                    header.extend(['spectral_features_'+str(i) for i in range(spectral_features.shape[0])])

            # Temporal features of the audio file.
            if build_temporal_features:
                cfg = tsfel.get_features_by_domain('temporal') 
                temporal_features = np.array(tsfel.time_series_features_extractor(cfg, x, fs=16000, window=300,verbose=0))[0]
                curr_feature = np.concatenate((curr_feature,temporal_features))
                if build_header:
                    header.extend(['temporal_features_'+str(i) for i in range(temporal_features.shape[0])])

            # Combining all the features.
            # curr_feature = np.concatenate((pid,stft_spectrogram,mfc_coefficients,melspectogram,mfe,mfcc,mfcc_cmvn,spectral_features,temporal_features))
            print('Feature extracted for',pid[0],' : ',curr_feature.shape[0])
            
            if build_header:
                all_features.append(header)
                build_header = False
                print('Header Added')

            all_features.append(curr_feature)

    print("\nFeature Extraction Done")
    return all_features

## Feature Extraction 

In [20]:
# Extracting the features from the audio files in the given directory.
generated_features = Audio_Feature_Extractor( directory='../1.Data/2.SpiroMask_Audio_Samples/',
                                        build_header=False,
                                        Apply_Bandpass_Filter=True,cutoff_low=3000,cutoff_high=5000,order=5,
                                        Apply_clipping=True,Auto_Clipping=True,
                                        Normalize=False)

1 / 76
Feature extracted for P0  :  711
2 / 76
Feature extracted for P1  :  711
3 / 76
Feature extracted for P10  :  711
4 / 76
Feature extracted for P11  :  711
5 / 76
Feature extracted for P12  :  711
6 / 76
Feature extracted for P13  :  711
7 / 76
Feature extracted for P14  :  711
8 / 76
Feature extracted for P15  :  711
9 / 76
Feature extracted for P16  :  711
10 / 76
Feature extracted for P17  :  711
11 / 76
Feature extracted for P18  :  711
12 / 76
Feature extracted for P19  :  711
13 / 76
Feature extracted for P2  :  711
14 / 76
Feature extracted for P20  :  711
15 / 76
Feature extracted for P21  :  711
16 / 76
Feature extracted for P22  :  711
17 / 76
Feature extracted for P23  :  711
18 / 76
Feature extracted for P24  :  711
19 / 76
Feature extracted for P25  :  711
20 / 76
Feature extracted for P26  :  711
21 / 76
Feature extracted for P27  :  711
22 / 76
Feature extracted for P28  :  711
23 / 76
Feature extracted for P29  :  711
24 / 76
Feature extracted for P3  :  711
25 / 

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1188
  warnings.warn(


Feature extracted for P30  :  711
26 / 76
Feature extracted for P31  :  711
27 / 76
Feature extracted for P32  :  711
28 / 76
Feature extracted for P33  :  711
29 / 76
Feature extracted for P34  :  711
30 / 76
Feature extracted for P35  :  711
31 / 76
Feature extracted for P36  :  711
32 / 76
Feature extracted for P37  :  711
33 / 76
Feature extracted for P38  :  711
34 / 76
Feature extracted for P39  :  711
35 / 76
Feature extracted for P4  :  711
36 / 76
Feature extracted for P40  :  711
37 / 76
Feature extracted for P41  :  711
38 / 76
Feature extracted for P42  :  711
39 / 76
Feature extracted for P43  :  711
40 / 76
Feature extracted for P44  :  711
41 / 76
Feature extracted for P45  :  711
42 / 76
Feature extracted for P46  :  711
43 / 76
Feature extracted for P47  :  711
44 / 76
Feature extracted for P48  :  711
45 / 76
Feature extracted for P49  :  711
46 / 76
Feature extracted for P5  :  711
47 / 76
Feature extracted for P50  :  711
48 / 76
Feature extracted for P51  :  711
49

In [21]:
np_features = np.array(generated_features)
print("Shape of the extracted features : ",np_features.shape)

Shape of the extracted features :  (76, 711)


## Merging Features and Labels (Ground Truth)

In [15]:
Name = "Autoclip_3000_5000_N95_FEATURES"    # Name of the file to be saved

GT = pd.read_csv('../1.Data/1.Ground_Truth/GroundTruth_Dataset.csv',delimiter=',',dtype='str',index_col=0).reset_index(drop=True)
feature_set = pd.DataFrame(generated_features)

final = pd.merge(feature_set,GT.iloc[:,[0,3,4,]],left_on=0,right_on='UID').drop(columns=['UID'])
Dataset = np.array(final)
print(Dataset.shape)

if not os.path.exists(('./1.Dataset/{}.npy').format(Name)):
    np.save(('./1.Dataset/{}.npy').format(Name),Dataset)
    print("Feature Set Saved")

(72, 713)
Feature Set Saved
